In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import math
import sklearn.decomposition
from skimage.transform import resize

%matplotlib inline

## Helper Functions

In [12]:
def load_data(path, col_name):
    """
    Load input data from matlab file.
    
    Parameters
    ----------
    path : String
        The relative path of matlab file    
    col_name : String
        Label of the input data within the
        file
    
    Returns
    -------
    N x D matrix
        Contains the resized image data
        where each column represents an
        image
    
    N x num_labels matrix
        Contains the one-hot encoded form
        of each data point's classification
    """
    resize_width = 17
    resize_height = 20
    
    ip = sio.loadmat(path)

    N = ip[col_name].shape[1] * ip[col_name][:, 0][0].shape[2]
    num_labels = ip[col_name].shape[1]

    size = (resize_height, resize_width)
    X = np.zeros((N, resize_height * resize_width))
    Y = np.zeros((N, num_labels))

    img_index = 0

    for i in range(num_labels):
        curr_class_data = ip[col_name][:,i][0]
        for j in range(curr_class_data.shape[2]):
            img_resized = resize(curr_class_data[:,:,j], size, mode='constant')
            X[img_index, :] = img_resized.flatten()
            Y[img_index, i] = 1
            img_index += 1
    
    return X, Y

def cross_entropy_loss_array(probs_array, y_onehot):
    indices = np.argmax(y_onehot, axis = 1).astype(int)
    predicted_probability = probs_array[np.arange(len(probs_array)), indices]
    log_preds = np.log(predicted_probability)
    loss = -1.0 * np.sum(log_preds) / len(log_preds)
    return loss

def regularization_L2_softmax_loss(reg_lambda, weight1, weight2):
    weight1_loss = 0.5 * reg_lambda * np.sum(weight1 * weight1)
    weight2_loss = 0.5 * reg_lambda * np.sum(weight2 * weight2)
    return weight1_loss + weight2_loss

def accuracy(predictions, labels):
    preds_correct_boolean =  np.argmax(predictions, 1) == np.argmax(labels, 1)
    correct_predictions = np.sum(preds_correct_boolean)
    accuracy = 100.0 * correct_predictions / predictions.shape[0]
    return accuracy

def softmax(output_array):
    logits_exp = np.exp(output_array)
    return logits_exp / np.sum(logits_exp, axis = 1, keepdims = True)

def tanh_derivative(data):
    """
    Computes the derivative after executing
    tanh function on the given data
    
    Parameters
    ----------
    data : N x D matrix
        The input data
    
    Returns
    -------
    N x D matrix
        The tanh derivative of the data
    """
    th = np.tanh(data)
    return 1 - th*th

def relu(data):
    """
    Applies the ReLU activation function on
    data passed as argument
    
    Parameters
    ----------
    data : N x D matrix
        The input data
       
    Returns
    -------
    N x D matrix
        The ReLU applied data
    """
    return np.maximum(data, 0)

def relu_derivative(data):
    """
    Applies the derivative of ReLU activation
    function on the data as argument
    
    Parameters
    ----------
    data : N x D matrix
        The input data
       
    Returns
    -------
    N x D matrix
        The ReLU derivative applied data
    """
    data[data <= 0] = 0
    data[data > 0] = 1
    return data

def sigmoid(Z):
    """
    Applies the sigmoid activation function: 
    1 / (1 + e ^ (-z)) where z represents
    each data point.
    
    Parameters
    ----------
    Z : N x D matrix
        The data
    
    Returns
    -------
    N x D matrix
        The data with sigmoid activation
        function applied on it
    """    
    return 1 / (1 + (np.exp(-Z)))

def sigmoid_derivative(Z):
    """
    Applies the derivative of sigmoid
    activation function: 1 / (1 + e ^ (-z))
    where z represents each data point.
    
    Parameters
    ----------
    Z : N x D matrix
        The data
    
    Returns
    -------
    N x D matrix
        The data with sigmoid activation
        function applied on it
    """
    partial = 1 / (1 + (np.exp(-Z)))
    return np.multiply(partial, (1 - partial))

def compute_mse(actual, reconstructed):
    """
    Computes the mean square error while
    reconstructing the input from encoded
    data.
    
    Parameters
    ----------
    actual : N x D matrix        
        Contains the actual input
    reconstructed : N x D matrix
        Constains the reconstructed points
        from decode layer
    
    Returns
    -------
    Double
        The calculated MSE
    """
    diff = actual - reconstructed
    return np.sum(np.sum(np.square(diff), axis=1), axis=0) / actual.shape[0]

## PCA

In [8]:
def pca(X, d):
    """
    Performs Principal Component Analysis on the
    given data to reduce its dimensions.
    
    Parameters
    ----------
    X : N x D matrix
        where N is the number of data points and
        D is the ambient dimension
    d : Integer
        Dimensionality of the low-dimensional
        representation
    
    Returns
    -------
    D x d matrix
        The basis of the low-dimensional subspace
    1 x d matrix
        The mean of the subspace
    d x N matrix
        The low dimensional representation of the
        input data
    """

    # Do mean normalization
    M_X = np.sum(X, axis = 0)
    M_X = M_X / X.shape[0]
    X = X - M_X

    # Find the correlation matrix
    C = np.dot(X.T, X) / X.shape[0]

    # Do eigenvalue decomposition and get hold of 
    # the eigenvalues (D) and eigenvectors (V) of 
    # covariance matrix
    D, V = np.linalg.eig(C)

    # Extract the top-d eigenvectors
    V = V[:, 0:d]
    
    # Represent data in this basis
    Y = np.dot(X, V)
    
    # Calculate the mean of low-dimensional space
    M_Y = np.sum(Y, axis=0) / Y.shape[0]
    
    return V.T, M_Y, Y.T

## Auto Encoder

In [14]:
class AutoEncoder:
    
    def __init__(self, f_activation, f_derivative, n_input, n_hidden, alpha = 0.1, bias = 1.0, l1_penalty = 0.01, mse_threshold = 3):
        
        # Initialize learning rate and bias
        self.alpha = alpha
        self.bias = bias
        self.l1_penalty = l1_penalty
        self.mse_threshold = mse_threshold
        
        # Store the activation function to be used by each neuron
        self.f_activation = f_activation
        self.f_derivative = f_derivative
        
        # Store the number of neurons in each layer
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_output = n_input
        
        # Initialize the weights and biases for hidden and output layers
        # Dimension of weight matrix for layer l: S[l+1] x S[l]
        self.W1 = np.random.normal(0, 1, [self.n_hidden, self.n_input + 1])
        self.W2 = np.random.normal(0, 1, [self.n_output, self.n_hidden + 1])
        
        # Initialize variables to store the input to neuron in each layer
        # Dimension of input vector for layer l: S[l]x1
        self.z2 = np.zeros((self.n_hidden, 1), dtype=float)
        self.z3 = np.zeros((self.n_output, 1), dtype=float)
        
        # Initialize variables to store the output for neurons in all layers
        # Dimension of output vector for layer l: (S[l]+1)x1
        self.a1 = np.zeros((self.n_input + 1, 1), dtype=float)
        self.a2 = np.zeros((self.n_hidden + 1, 1), dtype=float)
        self.a3 = np.zeros((self.n_output, 1), dtype=float)
        
        # Initialize variables to store the intermediate deltas
        # Dimension of delta vectors for layer l: S[l] x 1
        self.d2 = np.zeros((self.n_hidden), dtype=float)
        self.d3 = np.zeros((self.n_output), dtype=float)        
    
    def propogate_forward(self, curr_datapoint):
        """
        Performs forward propogation to calculate
        the inputs to neurons in the hidden and
        output layers.
        
        Parameters
        ----------
        curr_datapoint: a 1xD row vector which has the
            first data point
        """
        
        # Output from input layer will be the data passed as input
        # a1 -> D x 1 vector
        self.a1[:-1, 0] = curr_datapoint.T
        self.a1[-1:, 0] = self.bias
        
        # Hidden layer
        # z2 -> n_hidden x 1 vector
        # a2 -> (n_hidden + 1) x 1 vector
        self.z2 = np.dot(self.W1, self.a1)
        self.a2[:-1, :] = self.f_activation(self.z2)
        self.a2[-1:, :] = self.bias
        
        # Output layer
        # z3 -> n_output x 1 vector
        # a3 -> n_output x 1 vector
        self.z3 = np.dot(self.W2, self.a2)
        self.a3 = self.f_activation(self.z3)
    
    def propogate_back(self, y):
        """
        Performs backward propogation to calculate
        the partial derivative of cost function
        with respect to weights and bias terms 
        using intermediate values calculated during
        the forward propogation step.
        
        Parameters
        ----------
        y : a n_output x 1 vector
            which stores the actual class of
            the input
        """
        
        # Calculate delta from output layer
        # d3 -> n_output x 1 vector
        self.d3 = self.a3 - np.reshape(y, (self.n_output, 1))
        
        # Calculate delta from hidden layer
        # d2 -> n_hidden x 1 vector
        z2_derivative = self.f_derivative(self.z2)
        self.d2 = np.multiply(
                    np.dot(self.W2[:, :-1].T, self.d3), 
                    z2_derivative)
    
    def train(self, data):
        """
        Train the neural network with given data.
        
        Parameters
        ----------
        data : a N x D matrix
            where D is the number of dimensions &
            N is the number of data points.
            Contains the input data in the form
            of a matrix
        
        Returns
        -------
        (n_hidden + 1) x 1 vector
            Stores the weights for inputs of hidden 
            layer
        (n_output + 1) x 1 vector
            Stores the weights for inputs of output
            layer
        List<Double>
            The MSEs calculated for every 50th 
            iteration
        """
        
        N = data.shape[0]  # Each row represents a data point
        
        count = 0
        errors = []
        while(True):
            # Initialize variables to store the change in weight variables
            # Dimension of matrix for layer l: S[l+1] x S[l]
            D1 = np.matrix(np.zeros((self.n_hidden, self.n_input + 1), dtype=float))
            D2 = np.matrix(np.zeros((self.n_output, self.n_hidden + 1), dtype=float))

            for i in range(N):
                curr_datapoint = data[i, :]

                self.propogate_forward(curr_datapoint)
                self.propogate_back(curr_datapoint)
                
                D1[:, :-1] += np.dot(self.d2, self.a1[:-1, :].T)
                D1[:, -1:] += self.d2
                D2[:, :-1] += np.dot(self.d3, self.a2[:-1, :].T)
                D2[:, -1:] += self.d3
        
            # Update weights
            self.W1 -= self.alpha * ((1/N) * D1 + self.l1_penalty * self.W1)
            self.W2 -= self.alpha * ((1/N) * D2 + self.l1_penalty * self.W2)
            
            # Compute MSE for the generated weights
            ip_ones = np.ones((data.shape[0], 1))
            input_data = np.hstack((data, ip_ones))            
            encode_layer = np.dot(input_data, self.W1.T)
            encode_layer = self.f_activation(encode_layer)
            
            decode_ones = np.ones((encode_layer.shape[0], 1))
            decode_layer = np.hstack((encode_layer, decode_ones))
            decode_layer = np.dot(decode_layer, self.W2.T)
            
            reconstructed = self.f_activation(decode_layer)
            mse = compute_mse(data, reconstructed)
            errors.append(mse)
            
            # Check for convergence
            if (mse <= self.mse_threshold):
                print("Final MSE from training: ", mse)
                break
                
            if (count % 100 == 0):                
                print(count, ": ", mse)
            
            count += 1
        
        return self.W1, self.W2, encode_layer, errors

## Feed Forward Neural Network

In [10]:
class FFN:
    
    def __init__(self, f_activation, f_derivative, n_input, n_hidden, n_output, alpha=0.1, bias=1.0, l1_penalty=0.01):
        
        # Initialize learning rate and bias
        self.alpha = alpha
        self.bias = bias
        self.l1_penalty = l1_penalty
        
        # Store the activation function to be used by each neuron
        self.f_activation = f_activation
        self.f_derivative = f_derivative
        
        # Store the number of neurons in each layer
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        # Initialize the weights and biases for hidden and output layers
        # Dimension of weight matrix for layer l: S[l+1] x S[l]
        self.W1 = np.random.normal(0, 1, [self.n_hidden, self.n_input + 1])
        self.W2 = np.random.normal(0, 1, [self.n_output, self.n_hidden + 1])
        
        # Initialize variables to store the input to neuron in each layer
        # Dimension of input vector for layer l: S[l]x1
        self.z2 = np.zeros((self.n_hidden, 1), dtype=float)
        self.z3 = np.zeros((self.n_output, 1), dtype=float)
        
        # Initialize variables to store the output for neurons in all layers
        # Dimension of output vector for layer l: (S[l]+1)x1
        self.a1 = np.zeros((self.n_input + 1, 1), dtype=float)
        self.a2 = np.zeros((self.n_hidden + 1, 1), dtype=float)
        self.a3 = np.zeros((self.n_output, 1), dtype=float)
        
        # Initialize variables to store the intermediate deltas
        # Dimension of delta vectors for layer l: S[l] x 1
        self.d2 = np.zeros((self.n_hidden), dtype=float)
        self.d3 = np.zeros((self.n_output), dtype=float)        
    
    def propogate_forward(self, curr_datapoint):
        """
        Performs forward propogation to calculate
        the inputs to neurons in the hidden and
        output layers.
        
        Parameters
        ----------
        curr_datapoint: a 1xD row vector which has the
            first data point
        """
        
        # Output from input layer will be the data passed as input
        # a1 -> D x 1 vector
        self.a1[:-1, 0] = curr_datapoint.T
        self.a1[-1:, 0] = self.bias
        
        # Hidden layer
        # z2 -> n_hidden x 1 vector
        # a2 -> (n_hidden + 1) x 1 vector
        self.z2 = np.dot(self.W1, self.a1)
        self.a2[:-1, :] = self.f_activation(self.z2)
        self.a2[-1:, :] = self.bias
        
        # Output layer
        # z3 -> n_output x 1 vector
        # a3 -> n_output x 1 vector
        self.z3 = np.dot(self.W2, self.a2)
        self.a3 = self.f_activation(self.z3)
    
    def propogate_back(self, y):
        """
        Performs backward propogation to calculate
        the partial derivative of cost function
        with respect to weights and bias terms 
        using intermediate values calculated during
        the forward propogation step.
        
        Parameters
        ----------
        y : a n_output x 1 vector
            which stores the actual class of
            the input
        """
        
        # Calculate delta from output layer
        # d3 -> n_output x 1 vector
        self.d3 = self.a3 - np.reshape(y, (10, 1))
        
        # Calculate delta from hidden layer
        # d2 -> n_hidden x 1 vector
        z2_derivative = self.f_derivative(self.z2)
        self.d2 = np.multiply(
                    np.dot(self.W2[:, :-1].T, self.d3), 
                    z2_derivative)
    
    def train(self, data, labels, iterations=5001):
        """
        Train the neural network with given data.
        
        Parameters
        ----------
        data : a N x D matrix
            where D is the number of dimensions &
            N is the number of data points.
            Contains the input data in the form
            of a matrix
        labels : a N x n_output matrix
            Contains the true classes of each data
            point
        iterations : Integer [Optional field]            
            The number of iterations of training
        
        Returns
        -------
        (n_hidden + 1) x 1 vector
            Stores the weights for inputs of hidden 
            layer
        (n_output + 1) x 1 vector
            Stores the weights for inputs of output
            layer
        """
        
        N = data.shape[0]  # Each row represents a data point
                
        for count in range(iterations):
            # Initialize variables to store the change in weight variables
            # Dimension of matrix for layer l: S[l+1] x S[l]
            D1 = np.matrix(np.zeros((self.n_hidden, self.n_input + 1), dtype=float))
            D2 = np.matrix(np.zeros((self.n_output, self.n_hidden + 1), dtype=float))

            for i in range(N):
                curr_datapoint = data[i, :]
                curr_datapoint_label = labels[i, :]

                self.propogate_forward(curr_datapoint)
                self.propogate_back(curr_datapoint_label)
                
                D1[:, :-1] += np.dot(self.d2, self.a1[:-1, :].T)
                D1[:, -1:] += self.d2
                D2[:, :-1] += np.dot(self.d3, self.a2[:-1, :].T)
                D2[:, -1:] += self.d3
        
            # Update weights
            self.W1 -= self.alpha * ((1/N) * D1 + self.l1_penalty * self.W1)
            self.W2 -= self.alpha * ((1/N) * D2 + self.l1_penalty * self.W2)
            
            # Calculate prediction error
            input_layer = np.dot(data, self.W1[:, :-1].T)
            hidden_layer = self.f_activation(input_layer)
            scores = np.dot(hidden_layer, self.W2[:, :-1].T)
            probs = softmax(self.f_activation(scores))
            acc = accuracy(probs, labels)
            
            if (count % 1000 == 0):
                print("Iteration ", count, ": ", acc)
            
            # Check for convergence
            #fro_norm = np.linalg.norm(D2, 'fro')
            #print(fro_norm)
        
        return self.W1, self.W2

In [37]:
def execute_3a(f_activation, f_derivative, n_hidden, iterations=5001):
    X, Y = load_data("ExtYaleB10.mat", 'train')
    
    print("Initializind the Feed Forward Neural Network...")
    ffn = FFN(f_activation, f_derivative, X.shape[1], n_hidden, 10)
    
    print("Training the model...")
    wh, wo = ffn.train(X, Y, iterations)
    
    print("Running the model on test data...")
    test_data, test_labels = load_data("ExtYaleB10.mat", 'test')

    ip_ones = np.ones((test_data.shape[0], 1))
    test_data = np.hstack((test_data, ip_ones))
    input_layer = np.dot(test_data, wh.T)

    hidden_layer = f_activation(input_layer)
    hidden_ones = np.ones((hidden_layer.shape[0], 1))
    hidden_layer = np.hstack((hidden_layer, hidden_ones))

    scores = np.dot(hidden_layer, wo.T)
    scores = f_activation(scores)
    probs = softmax(scores)

    print('Test accuracy: {0}%'.format(accuracy(probs, test_labels)))

def execute_3d(f_activation, f_derivative, n_hidden, iterations=5001):
    X, Y = load_data("ExtYaleB10.mat", 'train')
    
    print("Reducing the dimension to 100 using PCA...")
    V_trn, M_trn, X_trn_low_dim = pca(X, 100)
    X_trn_low_dim = X_trn_low_dim.T
    
    print("Initializind the Feed Forward Neural Network...")
    ffn = FFN(f_activation, f_derivative, X_trn_low_dim.shape[1], n_hidden, 10)
    
    print("Training the model...")
    wh, wo = ffn.train(X_trn_low_dim, Y, iterations)
    
    print("Running the model on test data...")
    test_data, test_labels = load_data("ExtYaleB10.mat", 'test')
    
    print("Reducing dimensions of test data using PCA...")
    V_tst, M_tst, X_tst_low_dim = pca(test_data, 100)
    X_tst_low_dim = X_tst_low_dim.T

    ip_ones = np.ones((X_tst_low_dim.shape[0], 1))
    X_tst_low_dim = np.hstack((X_tst_low_dim, ip_ones))
    input_layer = np.dot(X_tst_low_dim, wh.T)

    hidden_layer = f_activation(input_layer)
    hidden_ones = np.ones((hidden_layer.shape[0], 1))
    hidden_layer = np.hstack((hidden_layer, hidden_ones))

    scores = np.dot(hidden_layer, wo.T)
    scores = f_activation(scores)
    probs = softmax(scores)

    print('Test accuracy: {0}%'.format(accuracy(probs, test_labels)))

def execute_3e(f_activation, f_derivative, n_hidden=10, alpha=0.1, bias=1.0, l1_penalty=0.01, mse_threshold=3, iterations=5000):

    f_activation=np.tanh
    f_derivative=tanh_derivative
    n_hidden=10
    alpha=0.1
    bias=1.0
    l1_penalty=0.01
    mse_threshold=3
    iterations=5000

    # Train the model
    X_trn, Y_trn = load_data("ExtYaleB10.mat", col_name='train')
    X_tst, Y_tst = load_data("ExtYaleB10.mat", col_name='test')

    # Generate the input and actual output matrices
    X = np.vstack((X_trn, X_tst))

    print("Initializind the Auto Encoder...")
    an = AutoEncoder(f_activation, f_derivative, X.shape[1], 100, alpha, bias, l1_penalty, mse_threshold)
    we, wd, ae, mse = an.train(X)
    
    # Train the Feed Forward Neural Network
    X_trn = ae[:X_trn.shape[0], :]

    print("Initializind the Feed Forward Neural Network...")
    ffn = FFN(relu, relu_derivative, n_input=X_trn.shape[1], n_hidden=50, n_output=10)

    print("Training the model...")
    wh, wo = ffn.train(X_trn, Y_trn, iterations)

    print("Running the model on test data...")
    X_tst = ae[X_trn.shape[0]:, :]

    ip_ones = np.ones((X_tst.shape[0], 1))
    X_tst = np.hstack((X_tst, ip_ones))
    input_layer = np.dot(X_tst, wh.T)

    hidden_layer = f_activation(input_layer)
    hidden_ones = np.ones((hidden_layer.shape[0], 1))
    hidden_layer = np.hstack((hidden_layer, hidden_ones))

    scores = np.dot(hidden_layer, wo.T)
    scores = f_activation(scores)
    probs = softmax(scores)

    print('Test accuracy: {0}%'.format(accuracy(probs, Y_tst)))

In [22]:
execute_3a(f_activation=np.tanh, f_derivative=tanh_derivative, n_hidden=50)

Initializind the Feed Forward Neural Network...
Training the model...
Iteration  0 :  9.0
Iteration  1000 :  33.2
Iteration  2000 :  96.0
Iteration  3000 :  99.6
Iteration  4000 :  98.0
Iteration  5000 :  99.2
Running the model on test data...
Test accuracy: 94.28571428571429%


In [ ]:
execute_3d(f_activation=np.tanh, f_derivative=tanh_derivative, n_hidden=50, iterations=2001)